In [1]:
import numpy as np
from numpy import linalg as LA
from scipy.linalg import expm
from scipy.sparse.linalg import LinearOperator, eigs
from ncon import ncon
from typing import Optional
import quimb.tensor as qtn


def doTEBD(hamAB: np.ndarray,
           hamBA: np.ndarray,
           A: np.ndarray,
           B: np.ndarray,
           sAB: np.ndarray,
           sBA: np.ndarray,
           chi: int,
           tau: float,
           evotype: Optional[str] = 'imag',
           numiter: Optional[int] = 1000,
           midsteps: Optional[int] = 10,
           E0: Optional[float] = 0.0):
    """
    Implementation of time evolution (real or imaginary) for MPS with 2-site unit
    cell (A-B), based on TEBD algorithm.
    Args:
    hamAB: nearest neighbor Hamiltonian coupling for A-B sites.
    hamBA: nearest neighbor Hamiltonian coupling for B-A sites.
    A: MPS tensor for A-sites of lattice.
    B: MPS tensor for B-sites of lattice.
    sAB: vector of weights for A-B links.
    sBA: vector of weights for B-A links.
    chi: maximum bond dimension of MPS.
    tau: time-step of evolution.
    evotype: set real (evotype='real') or imaginary (evotype='imag') evolution.
    numiter: number of time-step iterations to take.
    midsteps: number of time-steps between re-orthogonalization of the MPS.
    E0: specify the ground energy (if known).
    Returns:
    np.ndarray: MPS tensor for A-sites;
    np.ndarray: MPS tensor for B-sites;
    np.ndarray: vector sAB of weights for A-B links.
    np.ndarray: vector sBA of weights for B-A links.
    np.ndarray: two-site reduced density matrix rhoAB for A-B sites
    np.ndarray: two-site reduced density matrix rhoAB for B-A sites
    """
    # exponentiate Hamiltonian
    d = A.shape[1]
    if evotype == "real":
        gateAB = expm(1j * tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
        gateBA = expm(1j * tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)
    elif evotype == "imag":
        gateAB = expm(-tau * hamAB.reshape(d**2, d**2)).reshape(d, d, d, d)
        gateBA = expm(-tau * hamBA.reshape(d**2, d**2)).reshape(d, d, d, d)

    # initialize environment matrices
    sigBA = np.eye(A.shape[0]) / A.shape[0]
    muAB = np.eye(A.shape[2]) / A.shape[2]

    for k in range(numiter + 1):
        if np.mod(k, midsteps) == 0 or (k == numiter):
            """ Compute energy and display """

            # compute 2-site local reduced density matrices
            rhoAB, rhoBA = loc_density_MPS(A, sAB, B, sBA)

            # evaluate the energy
            hamAB_tensor = qtn.Tensor(hamAB, inds=('k1', 'k2', 'k3', 'k4'), tags=['hamAB'])
            rhoAB_tensor = qtn.Tensor(rhoAB, inds=('k1', 'k2', 'k3', 'k4'), tags=['rhoAB'])
            energyAB_tensor = hamAB_tensor & rhoAB_tensor
            energyAB = energyAB_tensor ^ ...
            
            hamBA_tensor = qtn.Tensor(hamBA, inds=('k1', 'k2', 'k3', 'k4'), tags=['hamBA'])
            rhoBA_tensor = qtn.Tensor(rhoBA, inds=('k1', 'k2', 'k3', 'k4'), tags=['rhoBA'])
            energyBA_tensor = hamBA_tensor & rhoBA_tensor
            energyBA = energyBA_tensor ^ ...
            
            energy = 0.5 * (energyAB + energyBA)

            chitemp = min(A.shape[0], B.shape[0])
            enDiff = energy - E0
            print('iteration: %d of %d, chi: %d, t-step: %f, energy: %f, '
                'energy error: %e' % (k, numiter, chitemp, tau, energy, enDiff))

        """ Do evolution of MPS through one time-step """
        if k < numiter:
            # apply gate to A-B link
            A, sAB, B = apply_gate_MPS(gateAB, A, sAB, B, sBA, chi)

            # apply gate to B-A link
            B, sBA, A = apply_gate_MPS(gateBA, B, sBA, A, sAB, chi)

    rhoAB, rhoBA = loc_density_MPS(A, sAB, B, sBA)
    
    return A, B, sAB, sBA, rhoAB, rhoBA


def apply_gate_MPS(gateAB, A, sAB, B, sBA, chi, stol=1e-7):
    """ apply a gate to an MPS across and a A-B link. Truncate the MPS back to
    some desired dimension chi"""

    # ensure singular values are above tolerance threshold
    sBA_trim = sBA * (sBA > stol) + stol * (sBA < stol)

    # contract gate into the MPS, then deompose composite tensor with SVD
    d = A.shape[1]
    chiBA = sBA_trim.shape[0]
    nshape = [d * chiBA, d * chiBA]
    
    sBA_1_tensor = qtn.Tensor(np.diag(sBA), inds=('f0', 'k1'), tags=['sBA', '1'])
    A_tensor = qtn.Tensor(A, inds=('k1', 'k2', 'k3'), tags=['A'])
    sAB_tensor = qtn.Tensor(np.diag(sAB), inds=('k3', 'k4'), tags=['sAB'])
    B_tensor = qtn.Tensor(B, inds=('k4', 'k5', 'k6'), tags=['B'])
    sBA_2_tensor = qtn.Tensor(np.diag(sBA), inds=('k6', 'f3'), tags=['sBA', '2'])
    gate_AB_tensor = qtn.Tensor(gateAB, inds=('f1', 'f2', 'k2', 'k5'), tags=['gateAB'])
    
    TN = sBA_1_tensor & gate_AB_tensor & A_tensor & sAB_tensor & B_tensor & sBA_2_tensor
    TNc = TN ^ ...
    x = TNc.data
    
    utemp, stemp, vhtemp = LA.svd(x.reshape(nshape), full_matrices=False)

    # truncate to reduced dimension
    chitemp = min(chi, len(stemp))
    utemp = utemp[:, range(chitemp)].reshape(sBA_trim.shape[0], d * chitemp)
    vhtemp = vhtemp[range(chitemp), :].reshape(chitemp * d, chiBA)

    # remove environment weights to form new MPS tensors A and B
    A = (np.diag(1 / sBA_trim) @ utemp).reshape(sBA_trim.shape[0], d, chitemp)
    B = (vhtemp @ np.diag(1 / sBA_trim)).reshape(chitemp, d, chiBA)

    # new weights
    sAB = stemp[range(chitemp)] / LA.norm(stemp[range(chitemp)])

    return A, sAB, B


def loc_density_MPS(A, sAB, B, sBA):
    """ Compute the local reduced density matrices from an MPS (assumend to be
    in canonical form)."""

    # recast singular weights into a matrix
    mAB = np.diag(sAB)
    mBA = np.diag(sBA)

    # contract MPS for local reduced density matrix (A-B)
    sBA_1_tensor = qtn.Tensor(np.diag(sBA**2), inds=('k3', 'k4'), tags=['sBA', '1'])
    A_tensor = qtn.Tensor(A, inds=('k3', 'f3', 'k1'), tags=['A'])
    A_conj_tensor = qtn.Tensor(A.conj(), inds=('k4', 'f1', 'k2'), tags=['A_conj'])
    mAB_1_tensor = qtn.Tensor(mAB, inds=('k1', 'k7'), tags=['mAB', '1'])
    mAB_2_tensor = qtn.Tensor(mAB, inds=('k2', 'k8'), tags=['mAB', '2'])
    B_tensor = qtn.Tensor(B, inds=('k7', 'f4', 'k5'), tags=['B'])
    B_conj_tensor = qtn.Tensor(B.conj(), inds=('k8', 'f2', 'k6'), tags=['B_conj'])
    sBA_2_tensor = qtn.Tensor(np.diag(sBA**2), inds=('k5', 'k6'), tags=['sBA', '2'])
    
    TN_rhoAB = (
        sBA_1_tensor & A_tensor & A_conj_tensor & mAB_1_tensor & mAB_2_tensor & B_tensor & B_conj_tensor & sBA_2_tensor
    )
    TN_rhoAB = TN_rhoAB ^ ...
    rhoAB = TN_rhoAB.transpose('f1', 'f2', 'f3', 'f4').data

    sAB_1_tensor = qtn.Tensor(np.diag(sAB**2), inds=('k3', 'k4'), tags=['sAB', '1'])
    B_tensor = qtn.Tensor(B, inds=('k3', 'f3', 'k1'), tags=['B'])
    B_conj_tensor = qtn.Tensor(B.conj(), inds=('k4', 'f1', 'k2'), tags=['B_conj'])
    mBA_1_tensor = qtn.Tensor(mAB, inds=('k1', 'k7'), tags=['mBA', '1'])
    mBA_2_tensor = qtn.Tensor(mAB, inds=('k2', 'k8'), tags=['mBA', '2'])
    A_tensor = qtn.Tensor(A, inds=('k7', 'f4', 'k5'), tags=['A'])
    A_conj_tensor = qtn.Tensor(A.conj(), inds=('k8', 'f2', 'k6'), tags=['A_conj'])
    sAB_2_tensor = qtn.Tensor(np.diag(sAB**2), inds=('k5', 'k6'), tags=['sAB', '2'])
    
    TN_rhoBA = (
        sAB_1_tensor & B_tensor & B_conj_tensor & mBA_1_tensor & mBA_2_tensor & A_tensor & A_conj_tensor & sAB_2_tensor
    )
    TN_rhoBA = TN_rhoBA ^ ...
    rhoBA = TN_rhoBA.transpose('f1', 'f2', 'f3', 'f4').data

    return rhoAB, rhoBA

In [2]:
import numpy as np
from ncon import ncon

""" Example 1: XX model """

# set bond dimensions and simulation options
chi = 16  # bond dimension
tau = 0.1  # timestep

numiter = 500  # number of timesteps
evotype = "imag"  # real or imaginary time evolution
E0 = -4 / np.pi  # specify exact ground energy (if known)
midsteps = int(1 / tau)  # timesteps between MPS re-orthogonalization

# define Hamiltonian (quantum XX model)
sX = np.array([[0, 1], [1, 0]])
sY = np.array([[0, -1j], [1j, 0]])
sZ = np.array([[1, 0], [0, -1]])
hamAB = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)
hamBA = (np.real(np.kron(sX, sX) + np.kron(sY, sY))).reshape(2, 2, 2, 2)

# initialize tensors
d = hamAB.shape[0]
sAB = np.ones(chi) / np.sqrt(chi)
sBA = np.ones(chi) / np.sqrt(chi)
A = np.random.rand(chi, d, chi)
B = np.random.rand(chi, d, chi)

""" Imaginary time evolution with TEBD """
# run TEBD routine
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# continute running TEBD routine with reduced timestep
tau = 0.01
numiter = 2000
midsteps = 100
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# continute running TEBD routine with reduced timestep and increased bond dim
chi = 32
tau = 0.001
numiter = 20000
midsteps = 1000
A, B, sAB, sBA, rhoAB, rhoBA = doTEBD(hamAB, hamBA, A, B, sAB, sBA, chi,
    tau, evotype=evotype, numiter=numiter, midsteps=midsteps, E0=E0)

# compare with exact results
energyMPS = np.real(0.5 * ncon([hamAB, rhoAB], [[1, 2, 3, 4], [1, 2, 3, 4]]) +
                    0.5 * ncon([hamBA, rhoBA], [[1, 2, 3, 4], [1, 2, 3, 4]]))
enErr = abs(energyMPS - E0)
print('Final results => Bond dim: %d, Energy: %f, Energy Error: %e' %
      (chi, energyMPS, enErr))

iteration: 0 of 500, chi: 16, t-step: 0.100000, energy: 4.250686, energy error: 5.523926e+00
iteration: 10 of 500, chi: 16, t-step: 0.100000, energy: -1.219391, energy error: 5.384824e-02
iteration: 20 of 500, chi: 16, t-step: 0.100000, energy: -1.234817, energy error: 3.842225e-02
iteration: 30 of 500, chi: 16, t-step: 0.100000, energy: -1.236906, energy error: 3.633382e-02
iteration: 40 of 500, chi: 16, t-step: 0.100000, energy: -1.237618, energy error: 3.562188e-02
iteration: 50 of 500, chi: 16, t-step: 0.100000, energy: -1.237936, energy error: 3.530368e-02
iteration: 60 of 500, chi: 16, t-step: 0.100000, energy: -1.238100, energy error: 3.513933e-02
iteration: 70 of 500, chi: 16, t-step: 0.100000, energy: -1.238193, energy error: 3.504627e-02
iteration: 80 of 500, chi: 16, t-step: 0.100000, energy: -1.238250, energy error: 3.498999e-02
iteration: 90 of 500, chi: 16, t-step: 0.100000, energy: -1.238285, energy error: 3.495418e-02
iteration: 100 of 500, chi: 16, t-step: 0.100000, en

iteration: 13000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272740, energy error: 4.992946e-04
iteration: 14000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272747, energy error: 4.923548e-04
iteration: 15000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272753, energy error: 4.864893e-04
iteration: 16000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272758, energy error: 4.814869e-04
iteration: 17000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272762, energy error: 4.771872e-04
iteration: 18000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272766, energy error: 4.734666e-04
iteration: 19000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272769, energy error: 4.702285e-04
iteration: 20000 of 20000, chi: 32, t-step: 0.001000, energy: -1.272772, energy error: 4.673959e-04
Final results => Bond dim: 32, Energy: -1.272772, Energy Error: 4.673959e-04
